In [1]:
import pandas as pd
from datetime import datetime
from textblob import TextBlob
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.offline import plot

In [2]:
# Read the CSV file
reviews_df = pd.read_csv('../../Warehouse/Reviews/app_reviews_1.csv')

In [3]:
# Filter out the rows without developer replies
replied_df = reviews_df.dropna(subset=['replyContent'])

In [4]:
# Convert at and repliedAt columns to datetime objects
replied_df['at'] = pd.to_datetime(replied_df['at'])
replied_df['repliedAt'] = pd.to_datetime(replied_df['repliedAt'])

C:\Users\Harsh Clean\AppData\Local\Temp\ipykernel_19004\671383597.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Harsh Clean\AppData\Local\Temp\ipykernel_19004\671383597.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [5]:
# Calculate response time in hours
replied_df['response_time'] = (replied_df['repliedAt'] - replied_df['at']).dt.total_seconds() / 3600

C:\Users\Harsh Clean\AppData\Local\Temp\ipykernel_19004\2207786185.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [6]:
# Calculate sentiment scores for review content and developer replies
replied_df['review_sentiment'] = replied_df['content'].apply(lambda x: TextBlob(x).sentiment.polarity)
replied_df['reply_sentiment'] = replied_df['replyContent'].apply(lambda x: TextBlob(x).sentiment.polarity)

C:\Users\Harsh Clean\AppData\Local\Temp\ipykernel_19004\3349007998.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Harsh Clean\AppData\Local\Temp\ipykernel_19004\3349007998.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [7]:
# Calculate average response time and sentiment scores
avg_response_time = replied_df['response_time'].mean()
avg_review_sentiment = replied_df['review_sentiment'].mean()
avg_reply_sentiment = replied_df['reply_sentiment'].mean()

In [8]:
# Identify trends in developer response times
replied_df['month_year'] = replied_df['repliedAt'].dt.to_period('M')
monthly_response_time = replied_df.groupby('month_year')['response_time'].mean()
print("\nMonthly Average Response Time:")
print(monthly_response_time)


Monthly Average Response Time:
month_year
2019-08      13.503722
2020-01    1375.897778
2020-03      76.843389
2020-04      30.618889
2020-06    2251.561178
2020-07      88.086833
2020-08     -21.332000
2020-09     347.500609
2020-10       8.456370
2020-11     -73.864651
2020-12      60.853544
2021-01       7.234214
2021-02      18.598472
2021-03    -204.653611
2021-04     -77.346998
2021-05     -77.583250
2021-06     -24.777056
2021-07       1.667847
2021-08       9.767956
2021-09     -18.941050
2021-10     -89.400673
2021-11    -112.740738
2021-12    -114.604724
2022-01    -123.550463
2022-02    -178.855890
2022-03    -143.481583
2022-04    -155.092091
2022-05    -155.751609
2022-06    -132.833506
2022-07    -115.151113
2022-08     -18.030669
2022-09     -31.732941
2022-10     103.312419
2022-11      48.921729
2022-12     -11.598366
2023-01      34.393888
2023-02       5.515155
2023-03     130.503980
2023-04    1378.015349
2023-05     576.702345
Freq: M, Name: response_time, dtype: 

C:\Users\Harsh Clean\AppData\Local\Temp\ipykernel_19004\990083051.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [9]:
fig = px.bar(x=[str(index) for index in monthly_response_time.index], y=monthly_response_time.values, 
             labels={'x': 'Month-Year', 'y': 'Response Time (hours)'}, 
             title='Monthly Average Response Time')
plot(fig, filename='../../fam-report-site/public/App-Analytics/AverageResponseTime.html', auto_open=False)
fig.show()

In [10]:
fig = px.scatter(replied_df, x='review_sentiment', y='reply_sentiment', 
                 opacity=0.5, labels={'review_sentiment': 'Review Sentiment', 
                                      'reply_sentiment': 'Reply Sentiment'},
                 title='Review Sentiment vs. Reply Sentiment')
plot(fig, filename='../../fam-report-site/public/App-Analytics/ReviewSentimentvsReplySentiment.html', auto_open=False)
fig.show()

In [11]:
# Correlate developer response sentiment with review sentiment
correlation = replied_df['review_sentiment'].corr(replied_df['reply_sentiment'])
print(f"\nCorrelation between Review Sentiment and Reply Sentiment: {correlation}")


Correlation between Review Sentiment and Reply Sentiment: 0.2951884402665913


In [12]:
monthly_response_time.to_csv('monthly_average_response_time.csv', header=True)
replied_df.to_csv('reviews_with_response_time_and_sentiment.csv', index=False)